In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import sqlite3 as sql
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn import utils


In [ ]:
stats_cnx = sql.connect('/content/drive/MyDrive/Colab Notebooks/Open-Sports-Data-NFL-main/Data/Combined Clean Historical Data/Historical_Fantasy_Stats.db')
stats = pd.read_sql_query('SELECT * FROM Historical_Fantasy_Points ', stats_cnx)
stats_cnx.close()
stats['season'] = pd.to_datetime(stats['season'], format='%Y')
stats_prep = stats.copy()

# Manipulating Data

In [ ]:
stats_prep['season'] = pd.to_datetime(stats_prep['season'], format='%Y')
stats_prep['nfanpts'] = stats_prep.groupby(['Name','player_id']).shift(-1)['fantast_pts']
stats_prep = stats_prep[stats_prep['nfanpts'].notna()]

# This is to take the players who record more then 40 fantasy points in a season.
stats_prep = stats_prep[stats_prep['nfanpts'] >= 40]
stats_prep

,Name,Position,player_id,season,fantast_pts,completions,attempts,passing_yards,passing_tds,interceptions,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,nfanpts
3,Donald Driver,WR,4541,2010-01-01,129.50,0,0,0,0,0,...,0.093617,0.391267,0.138957,0.142857,0.153439,0.152074,0.140907,0.139737,1.116601,117.50
6,Tony Gonzalez,TE,6101,2010-01-01,171.60,0,0,0,0,0,...,0.139824,0.421116,0.176107,0.214286,0.195000,0.197368,0.195197,0.183743,1.136915,209.50
7,Tony Gonzalez,TE,6101,2011-01-01,209.50,0,0,0,0,0,...,0.136058,0.448635,0.200458,0.241379,0.246512,0.245902,0.220919,0.208642,1.473064,234.00
8,Tony Gonzalez,TE,6101,2012-01-01,234.00,0,0,0,0,0,...,0.122271,0.456716,0.197076,0.250000,0.264228,0.262590,0.223538,0.207661,1.512195,218.90
10,Matt Hasselbeck,QB,7091,2010-01-01,148.04,266,444,3001,12,17,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,190.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,Josh Jacobs,RB,35700,2019-01-01,191.60,0,0,0,0,0,...,0.110507,0.088289,0.052432,0.000000,0.051282,0.045977,0.026216,0.041946,0.401937,231.30
5373,Miles Boykin,WR,35703,2019-01-01,50.80,0,0,0,0,0,...,0.010118,0.196293,0.073442,0.103448,0.072993,0.078313,0.088445,0.079443,0.578947,69.60
5375,Drew Lock,QB,35704,2019-01-01,68.00,100,156,1020,7,3,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,179.32
5377,Daniel Jones,QB,35710,2019-01-01,210.98,284,459,3027,24,12,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,178.02


In [ ]:
stats_prep.describe()

,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,target
count,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,...,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000
mean,36.522345,58.057401,424.205822,2.724067,1.373924,3.756868,24.625256,0.556376,0.278803,487.712177,...,0.113633,0.256268,0.115051,0.116933,0.114226,0.114471,0.116059,0.115534,0.831741,144.532751
std,100.976281,159.229348,1171.582192,7.832886,3.950939,10.497604,69.356918,1.689129,0.916626,1338.194134,...,0.088617,0.212742,0.095932,0.128760,0.097476,0.098102,0.105477,0.097633,0.704036,80.462713
min,0.000000,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-11.000000,...,-0.017544,-0.000483,-0.017223,0.000000,0.000000,0.000000,-0.008611,-0.013778,-0.096970,40.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.041635,0.077447,0.032282,0.000000,0.029183,0.028986,0.020298,0.029132,0.227824,78.370000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.106849,0.208637,0.099329,0.080000,0.095000,0.095238,0.094338,0.098755,0.705882,127.900000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.172868,0.417384,0.182703,0.200000,0.182243,0.184918,0.192427,0.185357,1.311879,196.050000
max,471.000000,727.000000,5477.000000,55.000000,27.000000,62.000000,420.000000,13.000000,8.000000,6377.000000,...,0.489141,0.999337,0.457247,1.000000,0.460123,0.467391,0.568404,0.470559,3.109278,471.200000


In [ ]:
stats_prep.rename(columns = {'nfanpts':'target'}, inplace = True)

In [ ]:
# unwanted_col = ['Name', 'Position', 'player_id','fantast_pts','rushing_epa', 
#            'receiving_2pt_conversions', 'special_teams_tds', 'rushing_2pt_conversions',
#            'rushing_fumbles_lost']

unwanted_col = ['Name', 'Position', 'player_id','fantast_pts']

stats_prep.drop(unwanted_col, axis=1, inplace=True)
predictors = list(stats_prep)
unwanted_predictor = {'season', 'target'}
predictors = [e for e in predictors if e not in unwanted_predictor]

In [ ]:
normalized_df=(stats_prep[predictors]-stats_prep[predictors].min())/(stats_prep[predictors].max()-stats_prep[predictors].min())

In [ ]:
stats_prep[predictors] = (stats_prep[predictors]-stats_prep[predictors].min())/(stats_prep[predictors].max()-stats_prep[predictors].min())
stats_prep.describe()

,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,...,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,target
count,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,...,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000,2439.000000
mean,0.077542,0.079859,0.078462,0.049528,0.050886,0.060595,0.058632,0.042798,0.034850,0.078070,...,0.258891,0.256797,0.278782,0.116933,0.248251,0.244914,0.216060,0.266988,0.289656,144.532751
std,0.214387,0.219022,0.213675,0.142416,0.146331,0.169316,0.165136,0.129933,0.114578,0.209486,...,0.174896,0.212780,0.202187,0.128760,0.211848,0.209893,0.182798,0.201581,0.219582,80.462713
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000
25%,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001722,...,0.116795,0.077943,0.104337,0.000000,0.063424,0.062016,0.050102,0.088595,0.101300,78.370000
50%,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001722,...,0.245504,0.209157,0.245646,0.080000,0.206467,0.203765,0.178418,0.232344,0.250402,127.900000
75%,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001722,...,0.375800,0.417942,0.421368,0.200000,0.396075,0.395639,0.348411,0.411150,0.439407,196.050000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,471.200000


In [ ]:
stats_prep = stats_prep.set_index(['season'])
stats_prep = stats_prep.sort_index()

In [ ]:
stats_prep.apply(pd.isnull).sum()

completions                    0
attempts                       0
passing_yards                  0
passing_tds                    0
interceptions                  0
sacks                          0
sack_yards                     0
sack_fumbles                   0
sack_fumbles_lost              0
passing_air_yards              0
passing_yards_after_catch      0
passing_first_downs            0
passing_epa                    0
passing_2pt_conversions        0
carries                        0
rushing_yards                  0
rushing_tds                    0
rushing_fumbles                0
rushing_fumbles_lost           0
rushing_first_downs            0
rushing_epa                    0
rushing_2pt_conversions        0
receptions                     0
targets                        0
receiving_yards                0
receiving_tds                  0
receiving_fumbles              0
receiving_fumbles_lost         0
receiving_air_yards            0
receiving_yards_after_catch    0
receiving_

In [ ]:
stats_prep['rtd_sh'] = stats_prep['rtd_sh'].fillna(stats_prep['rtd_sh'].mean())
stats_prep['dom'] = stats_prep['dom'].fillna(stats_prep['dom'].mean())
stats_prep['w8dom'] = stats_prep['w8dom'].fillna(stats_prep['w8dom'].mean())

In [ ]:
X = stats_prep[predictors].values
Y = stats_prep["target"]

In [86]:
X

,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,...,ay_sh,yac_sh,wopr,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa
season,,,,,,,,,,,,,,,,,,,,,
2010-01-01,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001722,...,0.401571,0.219389,0.391821,0.329168,0.142857,0.333474,0.325367,0.259124,0.316960,0.378502
2010-01-01,0.084926,0.090784,0.080066,0.054545,0.074074,0.112903,0.090476,0.076923,0.125,0.093300,...,0.100775,0.034625,0.000483,0.036299,0.000000,0.000000,0.000000,0.014924,0.028448,0.030244
2010-01-01,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001722,...,0.366382,0.348007,0.373546,0.356697,0.095238,0.347733,0.325309,0.229179,0.318871,0.427478
2010-01-01,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001722,...,0.649798,0.601805,0.665246,0.707160,0.416667,0.798635,0.799510,0.651796,0.726258,0.675059
2010-01-01,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001722,...,0.307855,0.094154,0.211799,0.135248,0.200000,0.197576,0.218320,0.228912,0.188581,0.136873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001722,...,0.147084,0.158792,0.101214,0.146755,0.000000,0.091831,0.083359,0.060337,0.115012,0.124065
2019-01-01,0.000000,0.000000,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001722,...,0.451645,0.394704,0.458171,0.482579,0.200000,0.493939,0.480304,0.371714,0.460784,0.482880
2019-01-01,0.000000,0.001376,0.001094,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.003601,...,0.810997,0.367789,0.646339,0.697306,0.230769,0.540040,0.526482,0.486659,0.641772,0.822237


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor(n_neighbors=34)

In [ ]:
knn_model.fit(X_train, y_train)


KNeighborsRegressor(n_neighbors=34)

In [ ]:
train_preds = knn_model.predict(X_train)

In [ ]:
rmse_train = r2_score(y_train, train_preds)
rmse_train

0.4000053413399185

In [ ]:
test_preds = knn_model.predict(X_test)
rmse_test = r2_score(y_test, test_preds)
rmse_test

0.37455857453373986

In [ ]:
Y

season
2010-01-01    117.50
2010-01-01     44.12
2010-01-01    124.70
2010-01-01    218.40
2010-01-01     62.10
               ...  
2019-01-01     45.00
2019-01-01    100.90
2019-01-01    328.60
2019-01-01    278.60
2019-01-01     80.70
Name: target, Length: 2439, dtype: float64

In [ ]:
combined = pd.concat([y_test, pd.Series(test_preds, index=y_test.index)], axis=1)
combined.columns = ["actual", "predictions"]

In [ ]:
combined

,actual,predictions
season,,
2015-01-01,215.3,85.882353
2013-01-01,160.1,127.381765
2018-01-01,141.8,149.544118
2016-01-01,171.5,222.750000
2013-01-01,60.9,94.452941
...,...,...
2014-01-01,89.1,226.250000
2018-01-01,259.4,137.032941
2011-01-01,158.3,95.641176


In [ ]:
import pickle

In [ ]:
filename = "knn_model.pkl"
pickle.dump(knn_model, open(filename, 'wb'))

# 2022 Predictions

In [ ]:
stats_cnx = sql.connect('/content/drive/MyDrive/Colab Notebooks/Open-Sports-Data-NFL-main/Data/Yearly Data as Database/Stats_with_names_2021.db')
stats_2021 = pd.read_sql_query('SELECT * FROM Stats_with_names_2021 ', stats_cnx)
stats_cnx.close()

In [ ]:
stats_2021['wopr'] = stats_2021['wopr_x'] + stats_2021['wopr_y']

In [ ]:
stats_2021['rtd_sh'] = stats_2021['rtd_sh'].fillna(stats_2021['rtd_sh'].mean())
stats_2021['dom'] = stats_2021['dom'].fillna(stats_2021['dom'].mean())
stats_2021['w8dom'] = stats_2021['w8dom'].fillna(stats_2021['w8dom'].mean())

In [ ]:
stats_2021[predictors] = (stats_2021[predictors]-stats_2021[predictors].min())/(stats_2021[predictors].max()-stats_2021[predictors].min())

In [ ]:
pred_test = knn_model.predict(stats_2021[predictors])

In [ ]:
stats_2021['rob_fpts_2022'] = pred_test

In [ ]:
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,279.628235,19596,2021,REG,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
1,Andy Lee,PN,91.846471,22824,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
2,Ben Roethlisberger,QB,210.277647,22924,2021,REG,0.804124,0.841446,0.703536,0.511628,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
3,Aaron Rodgers,QB,272.669412,23459,2021,REG,0.754639,0.738526,0.774078,0.860465,...,0.000000,0.001939,0.006196,0.000000,0.000000,0.000000,0.002086,0.004984,0.005212,0.003094
4,Ryan Fitzpatrick,QB,94.172353,23682,2021,REG,0.006186,0.008345,0.002445,0.000000,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,91.912353,37004,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.039469,0.047322,0.033600,0.000000,0.000000,0.000000,0.011311,0.027030,0.028768,0.014439
640,Tommy Tremble,TE,100.280000,37005,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.123737,0.165644,0.165483,0.090909,0.149254,0.189655,0.131967,0.178677,0.118827,0.188128
641,Tre' McKitty,TE,87.275294,37011,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.093049,0.054752,0.068233,0.000000,0.060606,0.066964,0.022970,0.054889,0.062250,0.031586
642,Trey Lance,QB,113.225294,37012,2021,REG,0.084536,0.098748,0.113431,0.116279,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000


In [ ]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,279.63,19596,2021,REG,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
1,Andy Lee,PN,91.85,22824,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
2,Ben Roethlisberger,QB,210.28,22924,2021,REG,0.804124,0.841446,0.703536,0.511628,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
3,Aaron Rodgers,QB,272.67,23459,2021,REG,0.754639,0.738526,0.774078,0.860465,...,0.000000,0.001939,0.006196,0.000000,0.000000,0.000000,0.002086,0.004984,0.005212,0.003094
4,Ryan Fitzpatrick,QB,94.17,23682,2021,REG,0.006186,0.008345,0.002445,0.000000,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,91.91,37004,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.039469,0.047322,0.033600,0.000000,0.000000,0.000000,0.011311,0.027030,0.028768,0.014439
640,Tommy Tremble,TE,100.28,37005,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.123737,0.165644,0.165483,0.090909,0.149254,0.189655,0.131967,0.178677,0.118827,0.188128
641,Tre' McKitty,TE,87.28,37011,2021,REG,0.000000,0.000000,0.000000,0.000000,...,0.093049,0.054752,0.068233,0.000000,0.060606,0.066964,0.022970,0.054889,0.062250,0.031586
642,Trey Lance,QB,113.23,37012,2021,REG,0.084536,0.098748,0.113431,0.116279,...,0.000000,0.000000,0.008495,0.000000,0.000000,0.000000,0.002860,0.006833,0.007427,0.000000


In [ ]:
predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)

In [ ]:
predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
0,Tom Brady,QB,279.63
3,Aaron Rodgers,QB,272.67
255,Patrick Mahomes,QB,270.68
14,Matthew Stafford,QB,267.52
538,Justin Herbert,QB,266.64
...,...,...,...
77,Keith Smith,RB,77.06
597,Brock Wright,TE,76.75
226,Isaiah Ford,WR,74.11
246,Greg Ward,WR,73.79


In [ ]:
top50 = predicted_fpts_2022.nlargest(50,'rob_fpts_2022')
top50['Position'].value_counts()

QB    24
WR    19
RB     5
TE     2
Name: Position, dtype: int64

In [ ]:
cnx = sql.connect("Predicted_fpts_2022_KNN_40.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out

In [ ]:
predicted_fpts_2022.to_sql('Predicted_fpts_2022_KNN_15', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database
cursor.close()
cnx.close()   

# Gridded KNN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {"n_neighbors": range(1, 41)}
gridsearch = GridSearchCV(KNeighborsRegressor(), parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 41)})

In [ ]:
gridsearch.best_params_

{'n_neighbors': 39}

In [ ]:
train_preds_grid = gridsearch.predict(X_train)
train_mse = mean_squared_error(y_train, train_preds_grid)
train_rmse = np.sqrt(train_mse)
test_preds_grid = gridsearch.predict(X_test)
test_mse = mean_squared_error(y_test, test_preds_grid)
test_rmse = np.sqrt(test_mse)

In [ ]:
train_rmse

63.50942858422268

In [ ]:
test_rmse

64.05183414644168

In [ ]:
grid_pred_test = gridsearch.predict(stats_2021[predictors])

In [ ]:
stats_2021['rob_fpts_2022'] = grid_pred_test

In [ ]:
cols = list(stats_2021)
cols.insert(2, cols.pop(cols.index('rob_fpts_2022')))
stats_2021 = stats_2021.loc[:, cols]
stats_2021['rob_fpts_2022'] = stats_2021['rob_fpts_2022'].apply(lambda x: round(x, 2))
stats_2021

,Name,Position,rob_fpts_2022,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,wopr
0,Tom Brady,QB,271.84,19596,2021,REG,485,719,5316.0,43,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Andy Lee,PN,79.07,22824,2021,REG,0,0,0.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Ben Roethlisberger,QB,202.06,22924,2021,REG,390,605,3740.0,22,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Aaron Rodgers,QB,232.19,23459,2021,REG,366,531,4115.0,37,...,0.000000,0.001939,-0.000923,0.000000,0.000000,0.000000,-0.000461,-0.000738,-0.007156,0.042417
4,Ryan Fitzpatrick,QB,79.07,23682,2021,REG,3,6,13.0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,Hunter Long,TE,77.59,37004,2021,REG,0,0,0.0,0,...,0.015625,0.047322,0.010076,0.000000,0.000000,0.000000,0.005038,0.008060,0.068966,0.197952
640,Tommy Tremble,TE,100.98,37005,2021,REG,0,0,0.0,0,...,0.048985,0.165644,0.063003,0.090909,0.074627,0.075862,0.076956,0.068584,0.360000,2.579061
641,Tre' McKitty,TE,93.06,37011,2021,REG,0,0,0.0,0,...,0.036836,0.054752,0.023974,0.000000,0.030303,0.026786,0.011987,0.019180,0.177165,0.433019
642,Trey Lance,QB,132.32,37012,2021,REG,41,71,603.0,5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
predicted_fpts_2022 = stats_2021.filter(['Name','Position','rob_fpts_2022'], axis=1)

In [ ]:
predicted_fpts_2022.sort_values(by=['rob_fpts_2022'], ascending=False)

,Name,Position,rob_fpts_2022
538,Justin Herbert,QB,277.17
87,Derek Carr,QB,276.45
255,Patrick Mahomes,QB,275.37
14,Matthew Stafford,QB,271.86
0,Tom Brady,QB,271.84
...,...,...,...
382,Khari Blasingame,RB,74.57
173,Derek Watt,RB,74.54
595,Adam Prentice,RB,74.54
480,Gabe Nabers,RB,74.20


In [ ]:
top50 = predicted_fpts_2022.nlargest(50,'rob_fpts_2022')
top50['Position'].value_counts()

QB    24
WR    17
RB     7
TE     2
Name: Position, dtype: int64

In [ ]:
cnx = sql.connect("Predicted_fpts_2022_KNN_Grid_Search.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out
predicted_fpts_2022.to_sql('Predicted_fpts_2022_KNN_Grid_Search', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database
cursor.close()
cnx.close()   